In [1]:
pip install kaggle


In [2]:
!kaggle datasets download -d balraj98/berkeley-segmentation-dataset-500-bsds500 -p sample_data


Dataset URL: https://www.kaggle.com/datasets/balraj98/berkeley-segmentation-dataset-500-bsds500
License(s): other
 95% 53.0M/56.0M [00:04<00:00, 17.9MB/s]
100% 56.0M/56.0M [00:04<00:00, 13.1MB/s]


In [3]:
import zipfile
import os
import numpy as np
import cv2
from scipy.io import loadmat
from sklearn.metrics import precision_score, recall_score
import matplotlib.pyplot as plt

# Define paths
zip_path = 'sample_data/berkeley-segmentation-dataset-500-bsds500.zip'  # Path to the downloaded zip file
extract_path = 'sample_data/bsds500'  # Path where the files will be extracted

# Function to extract files from ZIP archive
def extractZip(zip_path, extract_path):
    if not os.path.exists(extract_path):
        os.makedirs(extract_path)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Files extracted to {extract_path}")

# Function to apply Canny edge detection
def cannyEdgeDetection(image):
    edges = cv2.Canny(image, 100, 200)
    return edges

# Function to load ground truth from .mat file
def loadGroundTruth(mat_path):
    mat = loadmat(mat_path)
    ground_truth = mat['groundTruth'][0][0][0][0][1]  # Adjust this indexing based on your .mat file structure
    return ground_truth

# Function to calculate precision and recall with tolerance
def calculatePrecisionRecallWithTolerance(ground_truth, edges, tolerance=5):
    ground_truth = (ground_truth > 0).astype(np.uint8)  # Binarize ground truth
    edges = (edges > 0).astype(np.uint8)  # Binarize edges

    # Dilate ground truth to create a tolerance region
    kernel = np.ones((tolerance, tolerance), np.uint8)
    ground_truth_dilated = cv2.dilate(ground_truth, kernel, iterations=1)

    true_positive = np.sum((ground_truth_dilated == 1) & (edges == 1))
    false_positive = np.sum((ground_truth_dilated == 0) & (edges == 1))
    false_negative = np.sum((ground_truth == 1) & (edges == 0))

    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    return precision, recall

# Function to visualize images
def visualizeImages(image, edges, ground_truth):
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.title('Original Image')
    plt.imshow(image, cmap='gray')
    plt.subplot(1, 3, 2)
    plt.title('Canny Edges')
    plt.imshow(edges, cmap='gray')
    plt.subplot(1, 3, 3)
    plt.title('Ground Truth')
    plt.imshow(ground_truth, cmap='gray')
    plt.show()

# Extract files from ZIP
extractZip(zip_path, extract_path)

# Define paths for images and ground truth
image_path = os.path.join(extract_path, 'images/val')
ground_truth_path = os.path.join(extract_path, 'ground_truth/val')

# List files
image_files = [f for f in os.listdir(image_path) if f.endswith('.jpg')]
ground_truth_files = [f for f in os.listdir(ground_truth_path) if f.endswith('.mat')]

print(f"Number of images: {len(image_files)}")
print(f"Number of ground truths: {len(ground_truth_files)}")

# Process multiple images
for image_file in image_files[:100]:  # Process the first 10 images
    # Corresponding ground truth file (assume filenames match except for extension)
    gt_file = image_file.replace('.jpg', '.mat')

    if gt_file not in ground_truth_files:
        print(f"Missing ground truth for image {image_file}")
        continue

    # Load image
    image = cv2.imread(os.path.join(image_path, image_file), cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Could not load image {image_file}")
        continue

    # Apply Canny edge detection
    edges = cannyEdgeDetection(image)

    # Load ground truth
    gt_path = os.path.join(ground_truth_path, gt_file)
    ground_truth_data = loadGroundTruth(gt_path)

    # Convert ground truth to binary format
    ground_truth = (ground_truth_data > 0).astype(np.uint8)

    # Calculate precision and recall
    precision, recall = calculatePrecisionRecallWithTolerance(ground_truth, edges, tolerance=5)
    print(f'{image_file}: Precision={precision:.4f}, Recall={recall:.4f}')

    # Visualize images
   # visualizeImages(image, edges, ground_truth)


Files extracted to sample_data/bsds500
Number of images: 100
Number of ground truths: 100
196073.jpg: Precision=0.1417, Recall=0.7121
147091.jpg: Precision=0.2661, Recall=0.7257
299086.jpg: Precision=0.4212, Recall=0.5471
241048.jpg: Precision=0.1367, Recall=0.6607
220075.jpg: Precision=0.2288, Recall=0.4749
41033.jpg: Precision=0.2125, Recall=0.5678
304034.jpg: Precision=0.0584, Recall=0.7106
126007.jpg: Precision=0.2212, Recall=0.6162
12084.jpg: Precision=0.1202, Recall=0.6717
86000.jpg: Precision=0.1318, Recall=0.6451
241004.jpg: Precision=0.4331, Recall=0.6151
385039.jpg: Precision=0.1713, Recall=0.6709
3096.jpg: Precision=0.7099, Recall=0.6547
38092.jpg: Precision=0.2025, Recall=0.7212
295087.jpg: Precision=0.1425, Recall=0.6381
108005.jpg: Precision=0.1110, Recall=0.6151
42049.jpg: Precision=0.6131, Recall=0.6210
24077.jpg: Precision=0.3438, Recall=0.5853
14037.jpg: Precision=0.3103, Recall=0.4824
66053.jpg: Precision=0.1093, Recall=0.6283
108070.jpg: Precision=0.0345, Recall=0.6

In [4]:
import zipfile
import os
import numpy as np
import cv2
from scipy.io import loadmat
import matplotlib.pyplot as plt

# Define paths
zip_path = 'sample_data/berkeley-segmentation-dataset-500-bsds500.zip'  # Path to the downloaded zip file
extract_path = 'sample_data/bsds500'  # Path where the files will be extracted

# Function to extract files from ZIP archive
def extractZip(zip_path, extract_path):
    if not os.path.exists(extract_path):
        os.makedirs(extract_path)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Files extracted to {extract_path}")

# Function to apply Sobel edge detection
def sobelEdgeDetection(image):
    # Compute gradients
    grad_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)

    # Compute magnitude of gradients
    edges = np.sqrt(grad_x**2 + grad_y**2)

    # Convert to binary edge map
    edges = np.uint8(edges > np.max(edges) * 0.1)  # Adjust threshold as needed
    return edges

# Function to load ground truth from .mat file
def loadGroundTruth(mat_path):
    mat = loadmat(mat_path)
    ground_truth = mat['groundTruth'][0][0][0][0][1]  # Adjust this indexing based on your .mat file structure
    return ground_truth

# Function to calculate precision and recall with tolerance
def calculatePrecisionRecallWithTolerance(ground_truth, edges, tolerance=5):
    ground_truth = (ground_truth > 0).astype(np.uint8)  # Binarize ground truth
    edges = (edges > 0).astype(np.uint8)  # Binarize edges

    # Dilate ground truth to create a tolerance region
    kernel = np.ones((tolerance, tolerance), np.uint8)
    ground_truth_dilated = cv2.dilate(ground_truth, kernel, iterations=1)

    true_positive = np.sum((ground_truth_dilated == 1) & (edges == 1))
    false_positive = np.sum((ground_truth_dilated == 0) & (edges == 1))
    false_negative = np.sum((ground_truth == 1) & (edges == 0))

    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    return precision, recall

# Function to visualize images
def visualizeImages(image, edges, ground_truth):
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.title('Original Image')
    plt.imshow(image, cmap='gray')
    plt.subplot(1, 3, 2)
    plt.title('Sobel Edges')
    plt.imshow(edges, cmap='gray')
    plt.subplot(1, 3, 3)
    plt.title('Ground Truth')
    plt.imshow(ground_truth, cmap='gray')
    plt.show()

# Extract files from ZIP
extractZip(zip_path, extract_path)

# Define paths for images and ground truth
image_path = os.path.join(extract_path, 'images/val')
ground_truth_path = os.path.join(extract_path, 'ground_truth/val')

# List files
image_files = [f for f in os.listdir(image_path) if f.endswith('.jpg')]
ground_truth_files = [f for f in os.listdir(ground_truth_path) if f.endswith('.mat')]

print(f"Number of images: {len(image_files)}")
print(f"Number of ground truths: {len(ground_truth_files)}")

# Process multiple images
for image_file in image_files[:100]:  # Process the first 10 images
    # Corresponding ground truth file (assume filenames match except for extension)
    gt_file = image_file.replace('.jpg', '.mat')

    if gt_file not in ground_truth_files:
        print(f"Missing ground truth for image {image_file}")
        continue

    # Load image
    image = cv2.imread(os.path.join(image_path, image_file), cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Could not load image {image_file}")
        continue

    # Apply Sobel edge detection
    edges = sobelEdgeDetection(image)

    # Load ground truth
    gt_path = os.path.join(ground_truth_path, gt_file)
    ground_truth_data = loadGroundTruth(gt_path)

    # Convert ground truth to binary format
    ground_truth = (ground_truth_data > 0).astype(np.uint8)

    # Calculate precision and recall
    precision, recall = calculatePrecisionRecallWithTolerance(ground_truth, edges, tolerance=5)
    print(f'{image_file}: Precision={precision:.4f}, Recall={recall:.4f}')

    # Visualize images
    #visualizeImages(image, edges, ground_truth)


Files extracted to sample_data/bsds500
Number of images: 100
Number of ground truths: 100
196073.jpg: Precision=0.0737, Recall=0.9487
147091.jpg: Precision=0.3185, Recall=0.9338
299086.jpg: Precision=0.3371, Recall=0.8319
241048.jpg: Precision=0.1369, Recall=0.8851
220075.jpg: Precision=0.2269, Recall=0.7520
41033.jpg: Precision=0.1963, Recall=0.8766
304034.jpg: Precision=0.0613, Recall=0.9155
126007.jpg: Precision=0.2202, Recall=0.8365
12084.jpg: Precision=0.1144, Recall=0.9130
86000.jpg: Precision=0.1344, Recall=0.8842
241004.jpg: Precision=0.3547, Recall=0.8440
385039.jpg: Precision=0.1689, Recall=0.8750
3096.jpg: Precision=0.6592, Recall=0.9094
38092.jpg: Precision=0.2489, Recall=0.9349
295087.jpg: Precision=0.1451, Recall=0.8273
108005.jpg: Precision=0.1079, Recall=0.8510
42049.jpg: Precision=0.6366, Recall=0.8925
24077.jpg: Precision=0.3605, Recall=0.8778
14037.jpg: Precision=0.3462, Recall=0.7208
66053.jpg: Precision=0.1123, Recall=0.8958
108070.jpg: Precision=0.0322, Recall=0.8

In [5]:
import zipfile
import os
import numpy as np
import cv2
from scipy.io import loadmat
import matplotlib.pyplot as plt
from scipy.ndimage import convolve

# Define paths
zip_path = 'sample_data/berkeley-segmentation-dataset-500-bsds500.zip'  # Path to the downloaded zip file
extract_path = 'sample_data/bsds500'  # Path where the files will be extracted

# Function to extract files from ZIP archive
def extractZip(zip_path, extract_path):
    if not os.path.exists(extract_path):
        os.makedirs(extract_path)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Files extracted to {extract_path}")

# Function to apply Prewitt edge detection
def prewittEdgeDetection(image):
    # Define Prewitt kernels
    kernel_x = np.array([[1, 0, -1],
                         [1, 0, -1],
                         [1, 0, -1]], dtype=np.float32)

    kernel_y = np.array([[1, 1, 1],
                         [0, 0, 0],
                         [-1, -1, -1]], dtype=np.float32)

    # Apply Prewitt kernels
    grad_x = convolve(image, kernel_x)
    grad_y = convolve(image, kernel_y)

    # Compute magnitude of gradients
    edges = np.sqrt(grad_x**2 + grad_y**2)

    # Convert to binary edge map
    edges = np.uint8(edges > np.max(edges) * 0.1)  # Adjust threshold as needed
    return edges

# Function to load ground truth from .mat file
def loadGroundTruth(mat_path):
    mat = loadmat(mat_path)
    ground_truth = mat['groundTruth'][0][0][0][0][1]  # Adjust this indexing based on your .mat file structure
    return ground_truth

# Function to calculate precision and recall with tolerance
def calculatePrecisionRecallWithTolerance(ground_truth, edges, tolerance=5):
    ground_truth = (ground_truth > 0).astype(np.uint8)  # Binarize ground truth
    edges = (edges > 0).astype(np.uint8)  # Binarize edges

    # Dilate ground truth to create a tolerance region
    kernel = np.ones((tolerance, tolerance), np.uint8)
    ground_truth_dilated = cv2.dilate(ground_truth, kernel, iterations=1)

    true_positive = np.sum((ground_truth_dilated == 1) & (edges == 1))
    false_positive = np.sum((ground_truth_dilated == 0) & (edges == 1))
    false_negative = np.sum((ground_truth == 1) & (edges == 0))

    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    return precision, recall

# Function to visualize images
def visualizeImages(image, edges, ground_truth):
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.title('Original Image')
    plt.imshow(image, cmap='gray')
    plt.subplot(1, 3, 2)
    plt.title('Prewitt Edges')
    plt.imshow(edges, cmap='gray')
    plt.subplot(1, 3, 3)
    plt.title('Ground Truth')
    plt.imshow(ground_truth, cmap='gray')
    plt.show()

# Extract files from ZIP
extractZip(zip_path, extract_path)

# Define paths for images and ground truth
image_path = os.path.join(extract_path, 'images/val')
ground_truth_path = os.path.join(extract_path, 'ground_truth/val')

# List files
image_files = [f for f in os.listdir(image_path) if f.endswith('.jpg')]
ground_truth_files = [f for f in os.listdir(ground_truth_path) if f.endswith('.mat')]

print(f"Number of images: {len(image_files)}")
print(f"Number of ground truths: {len(ground_truth_files)}")

# Process multiple images
for image_file in image_files[:100]:  # Process the first 10 images
    # Corresponding ground truth file (assume filenames match except for extension)
    gt_file = image_file.replace('.jpg', '.mat')

    if gt_file not in ground_truth_files:
        print(f"Missing ground truth for image {image_file}")
        continue

    # Load image
    image = cv2.imread(os.path.join(image_path, image_file), cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Could not load image {image_file}")
        continue

    # Apply Prewitt edge detection
    edges = prewittEdgeDetection(image)

    # Load ground truth
    gt_path = os.path.join(ground_truth_path, gt_file)
    ground_truth_data = loadGroundTruth(gt_path)

    # Convert ground truth to binary format
    ground_truth = (ground_truth_data > 0).astype(np.uint8)

    # Calculate precision and recall
    precision, recall = calculatePrecisionRecallWithTolerance(ground_truth, edges, tolerance=5)
    print(f'{image_file}: Precision={precision:.4f}, Recall={recall:.4f}')

    # Visualize images
   # visualizeImages(image, edges, ground_truth)


Files extracted to sample_data/bsds500
Number of images: 100
Number of ground truths: 100
196073.jpg: Precision=0.0366, Recall=0.9969
147091.jpg: Precision=0.0927, Recall=0.9969
299086.jpg: Precision=0.1464, Recall=0.9970
241048.jpg: Precision=0.1129, Recall=0.9949
220075.jpg: Precision=0.1506, Recall=0.9963
41033.jpg: Precision=0.0753, Recall=0.9961
304034.jpg: Precision=0.0488, Recall=0.9958
126007.jpg: Precision=0.0972, Recall=0.9971
12084.jpg: Precision=0.0768, Recall=0.9982
86000.jpg: Precision=0.0856, Recall=0.9964
241004.jpg: Precision=0.1263, Recall=0.9965
385039.jpg: Precision=0.0964, Recall=0.9963
3096.jpg: Precision=0.0428, Recall=0.9969
38092.jpg: Precision=0.1284, Recall=0.9972
295087.jpg: Precision=0.1017, Recall=0.9964
108005.jpg: Precision=0.0784, Recall=0.9965
42049.jpg: Precision=0.1348, Recall=0.9967
24077.jpg: Precision=0.2229, Recall=0.9950
14037.jpg: Precision=0.1064, Recall=0.9938
66053.jpg: Precision=0.0789, Recall=0.9938
108070.jpg: Precision=0.0298, Recall=0.9